In [1]:
## 1- Intalar a biblioteca google-genai ##

%pip -q install google-genai

In [8]:
## 2- Configurar a API do google-genai ##

import os ##Importa o Módulo os
from google.colab import userdata ##Importa o Módulo userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY') #Armazena a APi como uma variável

In [9]:
## 3- Configura o cliente da SDK do Gemini

from google import genai #Importa o SDK

client = genai.Client() #Cria uma instância do cliente

MODEL_ID = "gemini-2.0-flash" #Define o modelo

In [10]:
# 4 -Instalar Framework ADK de agentes do google.
%pip -q install google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [12]:
# 5 Importar bibliotecas do ADK

from google.adk.agents import Agent # Permite criar agente personalizado.
from google.adk.runners import Runner # Permite controlar o fluxo da conversa usuário/IA.
from google.adk.sessions import InMemorySessionService # Cria sessões em memória para manter o contexto.
from google.adk.tools import google_search # Permite consulta na web via google.
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date # Para capturar data atual
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore") # Para suprimir avisos e manter o notebook limpo.

In [ ]:
# 6 Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [14]:
# 7 Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [22]:
##########################################
# --- Agente 1: Coordenador --- #
##########################################

def coordenador(topico, data_de_hoje):  # Cria uma função que utiliza topico e data_de_hoje como parametro.

    buscador = Agent(                      # Utiliza a função Agent do ADK
        name = "coordenador",          # Parametro nome
        model = "gemini-2.0-flash",         # Parametro nome
        description = "Agente que busca informações no google", # Parametro descrição da função
        tools = [google_search],       # Parametro de ferramentas do agente
        Instruction = """
        Você é um especialista em Recolocação e HeadHunter. Sua tarefa é utilizar a ferramenta (google_search)
        e preparar um relatório de vagas relevante e disponíveis para canditadura conforme o cargo abaixo.
        Liste as 10 vagas mais relevantes para o cargo indicado.
        Apresente vagas atuais e divulgadas em até 30 dias atrás.
        """                            # Parametro que cria a instrução para o agente_coordenador
   )

    entrada_do_agente_coordenador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"

    lancamentos = call_agent(agent= coordenador, message_text=entrada_do_agente_coordenador)

    return lancamentos